In [16]:
# import dependencies
import requests
import pandas as pd
from config import RecGov_API_Key
import time  
import json
import os
from flask import Flask, jsonify



In [17]:
# setup the base url for the API

base_url = 'https://ridb.recreation.gov/api/v1/'


In [18]:

# Create a function to fetch data from the Recreation.gov API
def fetch_ridb_data(api_key, endpoint, params=None, max_records=float('inf'), states = None):
    """Fetches data from the Recreation.gov API for various endpoints, handling pagination and rate limits.

    Args:
        api_key: Your Recreation.gov API key.
        endpoint: The API endpoint to hit (e.g., 'facilities', 'activities').
        params: A dictionary of query parameters.
        max_records: The maximum number of records to fetch in total. set to infinity by default.
         states: A list of states to pull data from.

    Returns:
        A list of dictionaries representing the data from the API.
    """

    headers = {
        'accept': 'application/json',
        'apikey': api_key
    }
    all_data = []
    limit = 50
    rate_limit_delay = 1.1
    if states is None:
        states = ["AZ"]
    for state in states:
      offset = 0
      while len(all_data) < max_records:
          current_params = {
            'limit': limit,
            'offset': offset,
             'state': state,
             'activity': 'Camping' # Added activity filter
            }
          if params:
              current_params.update(params)
          full_url = f"{base_url}{endpoint}"
          # Create the URL string and print it
          url_string = f"{full_url}?{requests.compat.urlencode(current_params)}"
          print(f"Request URL: {url_string}")

          try:
              response = requests.get(full_url, headers=headers, params=current_params)
              response.raise_for_status()
              data = response.json()

              if not data or "RECDATA" not in data:
                  print(f"No RECDATA found at offset {offset}. Ending for state {state}")
                  break

              all_data.extend(data["RECDATA"])
              offset += limit
              time.sleep(rate_limit_delay)

              if data.get("METADATA", {}).get("RESULTS", {}).get("CURRENT_COUNT", 0) < limit:
                    print(f"No more records available at offset {offset} for state {state}. Ending.")
                    break
          except requests.exceptions.RequestException as e:
            print(f"Error during API request at offset {offset}: {e}")
            break

          if len(all_data) >= max_records:
            print(f"Reached maximum number of records {max_records} for state {state}. Ending.")
            break

    return all_data



In [19]:
# Create a function to fetch related data from the Recreation.gov API
def fetch_ridb_related_data(api_key, endpoint, facility_id, params=None):
    """Fetches related data from the Recreation.gov API for a specific facility.

    Args:
        api_key: Your Recreation.gov API key.
        endpoint: The API endpoint to hit (e.g., 'activities', 'campsites').
        facility_id: The ID of the facility to fetch data for.
        params: A dictionary of query parameters.
    
    Returns:
        A list of dictionaries representing the data from the API.
    """
    headers = {
        'accept': 'application/json',
        'apikey': api_key
    }
    
    full_url = f"{base_url}facilities/{facility_id}/{endpoint}"
    
    try:
      response = requests.get(full_url, headers=headers, params=params)
      response.raise_for_status()
      data = response.json()
      if data and "RECDATA" in data:
        return data["RECDATA"]
      else:
        return [] # return empty list
    except requests.exceptions.RequestException as e:
      print(f"Error during API request: {e}")
      return []



In [20]:
# Create a function to create a pandas DataFrame from the extracted API data
def create_dataframe(data, data_key="RECDATA"):
    """Creates a pandas DataFrame from the extracted API data.

    Args:
        data: The list of dictionaries representing the data.
        data_key: The key to use when accessing the data (default is 'RECDATA')

    Returns:
        A pandas DataFrame containing the API data or an empty DataFrame if there was an error.
    """

    if data:
        return pd.DataFrame(data)
    else:
        return pd.DataFrame()
    


In [21]:
# Create a function to process the facilities data
def process_facilities_data(api_key, facilities_data):
    """Creates multiple DataFrames from the facilities data and related data.

    Args:
         api_key: Your Recreation.gov API key.
        facilities_data: The list of dictionaries representing the facilities from the API.

    Returns:
        A tuple containing the facilities_df, activities_df, campsites_df, and events_df.
    """
    if not facilities_data:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    facilities_df = create_dataframe(facilities_data)
    
    
    campsites_data = []
    
    permitted_equipment_data = []  # Initialize permitted equipment data
    campsite_attribute_db = []

    for facility in facilities_data:
        facility_id = facility.get('FacilityID')
        if facility_id:
             
            campsites = fetch_ridb_related_data(api_key, "campsites", facility_id)
            if campsites: #Check if there is data
              for camp in campsites:
                    camp['FacilityID'] = facility_id #assign the facility id as a foreign key to be used in tables
                    # Flatten Permitted Equipment data
                    for equipment in camp.get('PERMITTEDEQUIPMENT', []):
                         equipment['CampsiteID'] = camp.get('CampsiteID')
                         permitted_equipment_data.append(equipment) # adds permitted equipment data into the list
                    for attribute in camp.get('ATTRIBUTES', []):
                        attribute['CampsiteID'] = camp.get('CampsiteID')
                        campsite_attribute_db.append(attribute) #add the campsite attributes to the list
                    campsites_data.append(camp)
    
    
    facilities_df = facilities_df.drop(columns = ["ACTIVITY", "CAMPSITE", "EVENT"], errors = 'ignore') #Drop the columns that have blank data
    campsites_df = create_dataframe(campsites_data)
    permitted_equipment_df = create_dataframe(permitted_equipment_data) # Creates a DF for the permitted equipment
    campsite_attributes_df = create_dataframe(campsite_attribute_db) # Creates a DF for the campsite attributes
    
    # Merge Permitted Equipment Back to the Campsite Table (using a left join)
    campsites_df = pd.merge(campsites_df, permitted_equipment_df, on="CampsiteID", how='left', suffixes=('', '_permit'))
    campsites_df = pd.merge(campsites_df, campsite_attributes_df, on="CampsiteID", how='left', suffixes = ('', '_attrib'))

     # Drop nested columns from campsites table
    campsites_df = campsites_df.drop(columns = ['PERMITTEDEQUIPMENT', 'ATTRIBUTES'], errors ='ignore')
    
    return facilities_df, pd.DataFrame(), campsites_df, pd.DataFrame()
   

In [22]:
# Create function to output a DataFrame to a CSV file
def output_to_csv(df, filename):
    """Outputs a Pandas DataFrame to a CSV file.

    Args:
        df: The Pandas DataFrame to output.
        filename: The name of the CSV file to create.
    """
    if not df.empty:
      # Create the CSV Output Folder if it doesn't already exist
      output_dir = 'csv_output'
      if not os.path.exists(output_dir):
          os.makedirs(output_dir) # Creates a new directory

      output_file = os.path.join(output_dir, filename) # Create a path to output file
      df.to_csv(output_file, index=False)
      print(f"DataFrame successfully output to: {output_file}")
    else:
      print("Dataframe was empty and was not output to file.")


In [23]:
# Set up the API request, processes the Data and output Data Frames

if __name__ == "__main__":
    facilities_params = {
        'lastupdated': '10-01-2018',
    }
    facilities_endpoint = 'facilities'
    states = ["AZ", "OR", "UT"]
    # facilities_data = fetch_ridb_data(RecGov_API_Key, facilities_endpoint, facilities_params, 150)
    facilities_data = fetch_ridb_data(RecGov_API_Key, facilities_endpoint, facilities_params, states = states) # added list of states, and infinity value for max_records
    if facilities_data:
        facilities_df, activities_df, campsites_df, events_df = process_facilities_data(RecGov_API_Key, facilities_data)

        if not facilities_df.empty:
            print("Facilities DataFrame:")
            facilities_df.info()
            print(facilities_df.head())
            print(f"\nFacilities DataFrame has the following columns {facilities_df.columns.tolist()}")
            output_to_csv(facilities_df, "facilities.csv")
        else:
            print("Failed to create the facilities Dataframe")
        if not activities_df.empty:
            print("\nActivities DataFrame:")
            activities_df.info()
            print(activities_df.head())
            output_to_csv(activities_df, "activities.csv")
        else:
           print("\nFailed to create the activities Dataframe")
        if not campsites_df.empty:
            print("\nCampsites DataFrame:")
            campsites_df.info()
            print(campsites_df.head())
            output_to_csv(campsites_df, "campsites.csv")
        else:
           print("\nFailed to create the campsites Dataframe")
        if not events_df.empty:
           print("\nEvents DataFrame:")
           events_df.info()
           print(events_df.head())
           output_to_csv(events_df, "events.csv")
        else:
           print("\nFailed to create the events Dataframe")

Request URL: https://ridb.recreation.gov/api/v1/facilities?limit=50&offset=0&state=AZ&activity=Camping&lastupdated=10-01-2018
Request URL: https://ridb.recreation.gov/api/v1/facilities?limit=50&offset=50&state=AZ&activity=Camping&lastupdated=10-01-2018
Request URL: https://ridb.recreation.gov/api/v1/facilities?limit=50&offset=100&state=AZ&activity=Camping&lastupdated=10-01-2018
No more records available at offset 150 for state AZ. Ending.
Request URL: https://ridb.recreation.gov/api/v1/facilities?limit=50&offset=0&state=OR&activity=Camping&lastupdated=10-01-2018
Request URL: https://ridb.recreation.gov/api/v1/facilities?limit=50&offset=50&state=OR&activity=Camping&lastupdated=10-01-2018
Request URL: https://ridb.recreation.gov/api/v1/facilities?limit=50&offset=100&state=OR&activity=Camping&lastupdated=10-01-2018
Request URL: https://ridb.recreation.gov/api/v1/facilities?limit=50&offset=150&state=OR&activity=Camping&lastupdated=10-01-2018
Request URL: https://ridb.recreation.gov/api/v1/